In [1]:
!pip install bert_score
!pip install rouge_score
!pip install datasets

import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

from datasets import load_metric, Dataset
metric = load_metric("rouge")
from bert_score import score

import pandas as pd
import numpy as np
import nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=904929d58c420fd04e3aa5fb4e726ddf236bd7b9a207e1a967f5839ed1442c29
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


2024-05-19 14:53:44.234526: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 14:53:44.234628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 14:53:44.374791: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_34/1980692075.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rou

In [2]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
MIN_ALLOWED_SEQUENCE = 30
MAX_ALLOWED_SEQUENCE = 2048
BATCH_SIZE = 1
ACCUMULATION_STEPS = 4
LEARNING_RATE = 1e-6
EPOCHS = 20

data_collator = DataCollatorForSeq2Seq(tokenizer)

In [4]:
model.config.max_length = MAX_ALLOWED_SEQUENCE

In [5]:
def preprocess_function(examples):
    references = ["summarize: " + ref for ref in examples["reference"]]
    
    inputs = tokenizer(references, truncation=True, max_length=MAX_ALLOWED_SEQUENCE)
    targets = tokenizer(examples["summary"], truncation=True, max_length=MAX_ALLOWED_SEQUENCE)

    # Update examples with tokenized inputs and targets
    return {"input_ids": inputs.input_ids, "attention_mask": inputs.attention_mask, "labels": targets.input_ids}

In [6]:
train_df = pd.read_csv("train_processed.csv")
valid_df = pd.read_csv("validation_processed.csv")

train_df = train_df[train_df['reference_tokens_preprocessed'] < MAX_ALLOWED_SEQUENCE].reset_index(drop=True)
valid_df = valid_df[valid_df['reference_tokens_preprocessed'] < MAX_ALLOWED_SEQUENCE].reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/438 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Calculate ROUGE score
    rouge_result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    rouge_result = {key: value.mid.fmeasure * 100 for key, value in rouge_result.items()}
    
    # Calculate BERTScore
    P, R, F1 = score(decoded_preds, decoded_labels, lang='en', verbose=False)
    bertscore_result = {
        "bert_precision": P.mean().item() * 100,
        "bert_recall": R.mean().item() * 100,
        "bert_f1": F1.mean().item() * 100
    }
    
    # Calculate average prediction length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    gen_len = np.mean(prediction_lens)
    
    result = {**rouge_result, **bertscore_result, "gen_len": gen_len}
    
    return {k: round(v, 4) for k, v in result.items()}

In [8]:
args = Seq2SeqTrainingArguments(
    output_dir="./my_fine_tuned_t5_small_model",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    gradient_accumulation_steps=ACCUMULATION_STEPS,
    eval_accumulation_steps=ACCUMULATION_STEPS,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bert Precision,Bert Recall,Bert F1,Gen Len
0,No log,7.535876,12.289500,5.144900,8.961100,11.513300,85.387000,78.257300,81.656500,76.571400
2,No log,4.462596,12.838300,5.154300,9.334700,12.083200,85.051000,78.272100,81.512300,82.214300
4,6.610200,3.954201,13.478700,6.071500,9.887900,12.627200,85.453000,78.533200,81.838900,84.214300
6,6.610200,3.715705,13.445600,5.365900,9.654100,12.493300,84.434000,78.127400,81.145800,90.500000
8,6.610200,3.596324,13.250900,5.327500,9.368000,12.306500,84.620700,78.221600,81.287200,86.357100
10,4.256000,3.522705,14.125000,6.034500,10.256500,13.148500,84.682700,78.435400,81.430800,92.857100
12,4.256000,3.475759,14.291800,5.918900,10.097500,13.164500,84.597400,78.430000,81.387400,94.857100
14,3.996200,3.446184,14.423800,5.967700,10.207400,13.358400,84.510400,78.429400,81.348000,95.571400
16,3.996200,3.428485,14.106100,5.789400,10.064800,13.053400,84.427100,78.350800,81.266600,95.214300
18,3.915900,3.420532,14.129200,5.787200,10.091100,13.114000,84.428900,78.352400,81.268300,95.285700


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1197: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 2048}
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some non-default generation parameters are set in the model config. The

TrainOutput(global_step=2180, training_loss=4.62855342200043, metrics={'train_runtime': 2150.1089, 'train_samples_per_second': 4.074, 'train_steps_per_second': 1.014, 'total_flos': 2873064045969408.0, 'train_loss': 4.62855342200043, 'epoch': 19.91})

In [10]:
test_df = pd.read_csv("test_processed.csv")
test_df = test_df[test_df['reference_tokens_preprocessed'] < MAX_ALLOWED_SEQUENCE].reset_index(drop=True)

In [15]:
trained_results = []
device = torch.device("cuda")
model_trained = T5ForConditionalGeneration.from_pretrained("./my_fine_tuned_t5_small_model/checkpoint-2080").to(device)

#MIN_ALLOWED_SEQUENCE = 30
#MAX_ALLOWED_SEQUENCE = 2048
NUM_BEAMS = 4

for index, row in test_df.iterrows():
    inputs = tokenizer("summarize: " + row["reference"], max_length=MAX_ALLOWED_SEQUENCE, truncation=True, return_tensors="pt").to(device)
    outputs = model_trained.generate(**inputs, min_length=MIN_ALLOWED_SEQUENCE, max_length=MAX_ALLOWED_SEQUENCE,\
                                        num_beams=NUM_BEAMS, early_stopping=True)
    result_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(result_summary)

    P, R, F1 = score([result_summary], [row["summary"]], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    trained_results.append(F1.item())
    torch.cuda.empty_cache()
    
np.mean(trained_results) 

the commission ( eurostat ) shall adopt delegated act in accordance with the principle laid down in the interinstitutional agreement of 13 april 2016 on better law-making. the commission ( eurostat ) shall establish a formal expert group, composed of representative of all the member state and chaired by a representative of the commission ( eurostat ). the commission ( eurostat ) shall establish a formal expert group, composed of representative of all the member state and chaired by


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
article 1 subject matter the year 2021 shall be designated a ‘ european year of rail ’. the specific objective of the european year shall be to encourage and support the effort of the union, member state, regional and local authority, and other organisation to increase the share of passenger and freight moving by rail. the commission shall regularly convene meeting of the national contact person in order to coordinate the running of the european year.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.84
the european union recovery instrument ( the ‘ instrument ’ ) shall be carried out under specific union programme and in accordance with the objective of the instrument. the instrument shall be financed up to an amount of eur 750 000 million in 2018 price on the basis of the empowerment provided for in article 5 of the own resource decision. the measure shall be carried out under specific union programme and in accordance with the relevant union act laying down rule for those programme.


...

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.82
have adopted this regulation : article 1 obligation of the member state member state shall submit to the commission statistic on all the aquaculture activity conducted in freshwater and saltwater on their territory. article 2 definition 1. for the purpose of this regulation, the following definition shall apply : ( a ) ‘ community statistic ’ a defined in article 2 of regulation ( ec ) no 322/97 ; ( b ) ‘ capture-based aquaculture ’ mean the practice of collecting specimen from the wild and their subsequent


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.82


0.8139538235134549

In [12]:
del model_trained
untrained_results = []
device = torch.device("cuda")
model_untrained = T5ForConditionalGeneration.from_pretrained("google-t5/t5-large").to(device)
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-large")

for index, row in test_df.iterrows():
    inputs = tokenizer("summarize: " + row["reference"], max_length=MAX_ALLOWED_SEQUENCE, truncation=True, return_tensors="pt").to(device)
    outputs = model_untrained.generate(**inputs, min_length=MIN_ALLOWED_SEQUENCE, max_length=MAX_ALLOWED_SEQUENCE,\
                                        num_beams=NUM_BEAMS, early_stopping=True)
    result_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(result_summary)

    P, R, F1 = score([result_summary], [row["summary"]], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    untrained_results.append(F1.item())
    torch.cuda.empty_cache()
    
np.mean(untrained_results)   

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.82
a member state shall submit to the commission statistic on all the aquaculture activity conducted in freshwater and saltwater on their territory. member state shall use survey or other statistically validated method covering at least 90 % of the total production by volume. the remaining part of the total production may be estimated.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.84


0.8173354268074036

In [16]:
np.save("trained.npy", trained_results)
np.save("untrained.npy", untrained_results)